In [1]:
%load_ext autoreload
%autoreload 2

import spectral_unmixing_tools as el_spectral

In [44]:
pip install git+https://github.com/earthlab/cross-sensor-cal.git


  Cloning https://github.com/earthlab/cross-sensor-cal.git to /tmp/pip-req-build-3mcb9iw1
  Running command git clone --filter=blob:none --quiet https://github.com/earthlab/cross-sensor-cal.git /tmp/pip-req-build-3mcb9iw1
  Resolved https://github.com/earthlab/cross-sensor-cal.git to commit 8717d6b99deb35c43dc57fc2bec4882c2f254814
  Preparing metadata (setup.py) ... done
  Created wheel for EarthLabSpectral: filename=EarthLabSpectral-0.1-py3-none-any.whl size=1417 sha256=bebfc882dd13f60e077c2b288b2ae7dd343eae9db8bd57580f13cd335263fe64
  Stored in directory: /tmp/pip-ephem-wheel-cache-e_8ou5jp/wheels/d4/97/e8/c93ae5a8364c661ba5327b524ebe4ee01e41cfe582ac66f3d0
Successfully built EarthLabSpectral
Note: you may need to restart the kernel to use updated packages.


## Extract spectra as array

In [2]:
import numpy as np
import rasterio

class ENVIProcessor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.data = None  # This will hold the raster data array
        self.file_type = "envi"

    def load_data(self):
        """Loads the raster data from the file_path into self.data"""
        with rasterio.open(self.file_path) as src:
            self.data = src.read()  # Read all bands

    def get_chunk_from_extent(self, corrections=[], resample=False):
        self.load_data()  # Ensure data is loaded
        with rasterio.open(self.file_path) as src:
            bounds = src.bounds
            width, height = src.width, src.height
            col_start, line_start = 0, 0
            col_end, line_end = width, height

            # Assuming self.data is a 3D numpy array with dimensions [bands, rows, cols]
            chunk = self.data[:, line_start:line_end, col_start:col_end]

            # Apply any processing to chunk here...
            # For example, to demonstrate, flip chunk vertically
            chunk = np.flip(chunk, axis=1)

            return chunk

def load_and_combine_rasters(raster_paths):
    """
    Loads and combines raster data from a list of file paths.
    """
    chunks = []
    for path in raster_paths:
        processor = ENVIProcessor(path)
        chunk = processor.get_chunk_from_extent(corrections=['some_correction'], resample=False)
        chunks.append(chunk)

    combined_array = np.concatenate(chunks, axis=0)  # Combine along the first axis (bands)
    return combined_array

# Provided raster paths
raster_paths = [
    "NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200801_161441_reflectance/NEON_D13_NIWO_DP1_20200801_161441_reflectanceNEON_D13_NIWO_DP1_20200801_161441_reflectance__envi_resample_Landsat_5_TM.img",
    "NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200801_161441_reflectance/NEON_D13_NIWO_DP1_20200801_161441_reflectanceNEON_D13_NIWO_DP1_20200801_161441_reflectance__envi_resample_Landsat_7_ETMplus.img",
    "NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200801_161441_reflectance/NEON_D13_NIWO_DP1_20200801_161441_reflectanceNEON_D13_NIWO_DP1_20200801_161441_reflectance__envi_resample_Landsat_8_OLI.img",
    "NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200801_161441_reflectance/NEON_D13_NIWO_DP1_20200801_161441_reflectanceNEON_D13_NIWO_DP1_20200801_161441_reflectance__envi_resample_Landsat_9_OLI-2.img",
    "NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200801_161441_reflectance/NEON_D13_NIWO_DP1_20200801_161441_reflectanceNEON_D13_NIWO_DP1_20200801_161441_reflectance__envi",
    "NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200801_161441_reflectance/NEON_D13_NIWO_DP1_20200801_161441_reflectance"
]

# Ensure paths are updated correctly, especially for Landsat 7, 8, and 9 as provided paths are duplicates of Landsat 5
combined_array = load_and_combine_rasters(raster_paths)
combined_array.shape

(876, 10782, 1071)

In [3]:
import pandas as pd
import numpy as np

def process_and_flatten_array(array, landsat_versions=[5, 7, 8, 9], bands_per_landsat=6):
    """
    Processes a 3D numpy array to a DataFrame, renames columns, and adds Pixel_id.
    
    Parameters:
    - array: A 3D numpy array of shape (bands, rows, cols).
    - landsat_versions: A list of Landsat versions to use for naming.
    - bands_per_landsat: Number of bands per Landsat version.
    
    Returns:
    - A pandas DataFrame with processed and renamed columns and added Pixel_id.
    """
    if len(array.shape) != 3:
        raise ValueError("Input array must be 3-dimensional.")
    
    # Flatten the array
    bands, rows, cols = array.shape
    reshaped_array = array.reshape(bands, -1).T  # Transpose to make bands as columns
    pixel_indices = np.indices((rows, cols)).reshape(2, -1).T  # Row and col indices
    
    # Create DataFrame
    df = pd.DataFrame(reshaped_array, columns=[f'Band_{i+1}' for i in range(bands)])
    df.insert(0, 'Pixel_Col', pixel_indices[:, 1])
    df.insert(0, 'Pixel_Row', pixel_indices[:, 0])
    df.insert(0, 'Pixel_id', np.arange(len(df)))

    # Renaming columns
    total_bands = bands
    original_and_corrected_bands = total_bands - bands_per_landsat * len(landsat_versions)
    band_per_version = original_and_corrected_bands // 2  # Assuming equal original and corrected bands
    
    new_names = ([f"Original_band_{i}" for i in range(1, band_per_version + 1)] +
                 [f"Corrected_band_{i}" for i in range(1, band_per_version + 1)])
    
    for version in landsat_versions:
        new_names.extend([f"Landsat_{version}_band_{i}" for i in range(1, bands_per_landsat + 1)])
    
    # Apply new column names for band columns
    df.columns = ['Pixel_id', 'Pixel_Row', 'Pixel_Col'] + new_names

    return df

# Example usage with a hypothetical array shape
# Assume 'combined_array' is your loaded and combined 3D numpy array
# combined_array = np.random.rand(426*2 + 4*6, 100, 100)  # Example array

# Process and flatten the array
df_processed = process_and_flatten_array(combined_array)

# Now 'df_processed' is the DataFrame with renamed columns and added Pixel_id
df_processed

,Pixel_id,Pixel_Row,Pixel_Col,Original_band_1,Original_band_2,Original_band_3,Original_band_4,Original_band_5,Original_band_6,Original_band_7,...,Landsat_8_band_3,Landsat_8_band_4,Landsat_8_band_5,Landsat_8_band_6,Landsat_9_band_1,Landsat_9_band_2,Landsat_9_band_3,Landsat_9_band_4,Landsat_9_band_5,Landsat_9_band_6
0,0,0,0,-9998.999998,-9999.0,-9999.0,-9999.0,-9999.0,-9998.857661,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1,1,0,1,-9998.999998,-9999.0,-9999.0,-9999.0,-9999.0,-9998.857661,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2,2,0,2,-9998.999998,-9999.0,-9999.0,-9999.0,-9999.0,-9998.857661,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
3,3,0,3,-9998.999998,-9999.0,-9999.0,-9999.0,-9999.0,-9998.857661,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
4,4,0,4,-9998.999998,-9999.0,-9999.0,-9999.0,-9999.0,-9998.857661,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11547517,11547517,10781,1066,-9998.999998,-9999.0,-9999.0,-9999.0,-9999.0,-9998.857661,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
11547518,11547518,10781,1067,-9998.999998,-9999.0,-9999.0,-9999.0,-9999.0,-9998.857661,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
11547519,11547519,10781,1068,-9998.999998,-9999.0,-9999.0,-9999.0,-9999.0,-9998.857661,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
11547520,11547520,10781,1069,-9998.999998,-9999.0,-9999.0,-9999.0,-9999.0,-9998.857661,-9999.0,...,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0


## flatten array into a 2D df


In [ ]:
import pandas as pd
import numpy as np

def clean_data_and_write_to_csv(df, output_csv_path, chunk_size=100000):
    """
    Cleans the DataFrame in chunks to minimize memory usage and writes the cleaned
    chunks directly to a CSV file to avoid memory overload. It replaces values approximately
    equal to -9999 (within a tolerance of 1) with NaN in columns not starting with 'Pixel', and
    then drops rows where all such columns are NaN.
    
    Parameters:
    - df: pandas DataFrame to clean.
    - output_csv_path: Path to the output CSV file.
    - chunk_size: Number of rows in each chunk.
    
    Returns:
    - None. The cleaned data is written directly to the specified CSV file.
    """
    total_rows = df.shape[0]
    num_chunks = (total_rows // chunk_size) + (1 if total_rows % chunk_size else 0)

    print(f"Starting cleaning process, total rows: {total_rows}, chunk size: {chunk_size}, total chunks: {num_chunks}")

    # Initialize CSV file writing
    first_chunk = True

    for i, start_row in enumerate(range(0, total_rows, chunk_size)):
        chunk = df.iloc[start_row:start_row + chunk_size].copy()

        # Replace values close to -9999 with NaN
        for col in chunk.columns:
            if not col.startswith('Pixel'):
                chunk[col] = np.where(np.isclose(chunk[col], -9999, atol=1), np.nan, chunk[col])

        # Drop rows where all non-'Pixel' columns are NaN
        non_pixel_columns = [col for col in chunk.columns if not col.startswith('Pixel')]
        chunk.dropna(subset=non_pixel_columns, how='all', inplace=True)
        
        # Write processed chunk to CSV
        if first_chunk:
            chunk.to_csv(output_csv_path, mode='w', header=True, index=False)
            first_chunk = False
        else:
            chunk.to_csv(output_csv_path, mode='a', header=False, index=False)
        
        print(f"Processed and wrote chunk {i+1}/{num_chunks} to CSV.")

    print("Cleaning process completed and data written to CSV.")

# Specify the output CSV path
output_csv_path = 'NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200801_161441_reflectance_active_pixels.csv'

# Apply the function to your DataFrame
clean_data_and_write_to_csv(df_processed, output_csv_path)


Starting cleaning process, total rows: 11547522, chunk size: 100000, total chunks: 116
Processed and wrote chunk 1/116 to CSV.
Processed and wrote chunk 2/116 to CSV.
Processed and wrote chunk 3/116 to CSV.
Processed and wrote chunk 4/116 to CSV.
Processed and wrote chunk 5/116 to CSV.
Processed and wrote chunk 6/116 to CSV.
Processed and wrote chunk 7/116 to CSV.


## Reshape data frame and change labels

In [55]:
import numpy as np

def iterate_flatten_melt_array(array):
    """
    Generator to iterate over a 3D numpy array and yield "melted" data.
    
    Parameters:
    - array: A 3D numpy array of shape (bands, rows, cols).
    
    Yields:
    - Tuple of (Pixel_Row, Pixel_Col, Band_ID, Wavelength) for each pixel-band combination.
    """
    bands, rows, cols = array.shape
    
    for band in range(bands):
        for row in range(rows):
            for col in range(cols):
                yield (row, col, f'Band_{band+1}', array[band, row, col])

# Example usage
#chunk = np.random.rand(426, 11138, 1031)  # Replace with your actual data

# To demonstrate or test the generator, you can iterate through a small portion of it
for i, data_point in enumerate(iterate_flatten_melt_array(chunk)):
    print(data_point)
    if i > 100:  # Adjust this condition to control how many items you want to print
        break


(0, 0, 'Band_1', -9998.999997820287)
(0, 1, 'Band_1', -9998.999997820287)
(0, 2, 'Band_1', -9998.999997820287)
(0, 3, 'Band_1', -9998.999997820287)
(0, 4, 'Band_1', -9998.999997820287)
(0, 5, 'Band_1', -9998.999997820287)
(0, 6, 'Band_1', -9998.999997820287)
(0, 7, 'Band_1', -9998.999997820287)
(0, 8, 'Band_1', -9998.999997820287)
(0, 9, 'Band_1', -9998.999997820287)
(0, 10, 'Band_1', -9998.999997820287)
(0, 11, 'Band_1', -9998.999997820287)
(0, 12, 'Band_1', -9998.999997820287)
(0, 13, 'Band_1', -9998.999997820287)
(0, 14, 'Band_1', -9998.999997820287)
(0, 15, 'Band_1', -9998.999997820287)
(0, 16, 'Band_1', -9998.999997820287)
(0, 17, 'Band_1', -9998.999997820287)
(0, 18, 'Band_1', -9998.999997820287)
(0, 19, 'Band_1', -9998.999997820287)
(0, 20, 'Band_1', -9998.999997820287)
(0, 21, 'Band_1', -9998.999997820287)
(0, 22, 'Band_1', -9998.999997820287)
(0, 23, 'Band_1', -9998.999997820287)
(0, 24, 'Band_1', -9998.999997820287)
(0, 25, 'Band_1', -9998.999997820287)
(0, 26, 'Band_1', -999

In [ ]:
import csv

# Open a CSV file for writing
with open('melted_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Pixel_Row', 'Pixel_Col', 'Band_ID', 'Wavelength'])  # Write header

    # Write each data point
    for data_point in iterate_flatten_melt_array(chunk):
        writer.writerow(data_point)
import numpy as np
import pandas as pd

def batch_flatten_melt_array(array, batch_size=1000000):
    """
    Generator to iterate over a 3D numpy array and yield batches of "melted" data.
    
    Parameters:
    - array: A 3D numpy array of shape (bands, rows, cols).
    - batch_size: The number of rows in each batch.
    
    Yields:
    - A DataFrame containing a batch of melted data.
    """
    bands, rows, cols = array.shape
    total_pixels = rows * cols
    num_batches = (total_pixels + batch_size - 1) // batch_size  # Ceiling division to get the number of batches
    
    for batch in range(num_batches):
        batch_data = []
        start_index = batch * batch_size
        end_index = min(start_index + batch_size, total_pixels)
        
        for index in range(start_index, end_index):
            row = index // cols
            col = index % cols
            for band in range(bands):
                batch_data.append((row, col, f'Band_{band+1}', array[band, row, col]))
                
        batch_df = pd.DataFrame(batch_data, columns=['Pixel_Row', 'Pixel_Col', 'Band_ID', 'Wavelength'])
        yield batch_df

# Example usage
#chunk = np.random.rand(426, 11138, 1031)  # Replace with your actual data

# Iterate through each batch and process
for i, batch_df in enumerate(batch_flatten_melt_array(chunk)):
    print(f"Processing batch {i+1}")
    # Process the batch_df here
    # For example, you could save each batch to a separate CSV file
    batch_df.to_csv(f'melted_data_batch_{i+1}.csv', index=False)
   # if i == 0:  # For demonstration, break after processing the first batch
    #    break


## Exract by polygon

In [21]:
import rasterio
import numpy as np
import geopandas as gpd
from rasterio.windows import from_bounds
from shapely.geometry import box

class ENVIProcessor:
    def __init__(self, raster_path, polygons_path):
        self.raster_path = raster_path
        self.polygons_path = polygons_path
        self.polygons = None
        self.raster_meta = None
        
    def load_polygons(self):
        """Loads the polygons and ensures they are in the same CRS as the raster."""
        with rasterio.open(self.raster_path) as src:
            self.raster_meta = src.meta
            self.polygons = gpd.read_file(self.polygons_path)
            self.polygons = self.polygons.to_crs(src.crs)
    
    def extract_data_by_polygons(self):
        """Extracts the row and col indices from the raster for each polygon and appends all attributes from the polygons."""
        self.load_polygons()  # Load polygons and ensure CRS match
        
        all_data = []
        with rasterio.open(self.raster_path) as src:
            raster_bounds = src.bounds
            raster_box = box(*raster_bounds)
            
            for _, poly in self.polygons.iterrows():
                geom = poly.geometry
                # Skip invalid or empty geometries or those that do not intersect with the raster
                if geom is None or geom.is_empty or not geom.intersects(raster_box):
                    continue
                
                window = from_bounds(*geom.bounds, transform=src.transform)
                # Skip windows that are completely outside the raster bounds
                if window.width <= 0 or window.height <= 0:
                    continue
                
                # Convert window offsets to integers
                row_off = int(window.row_off)
                col_off = int(window.col_off)
                # Extract the rows and cols from the window
                rows = range(row_off, row_off + int(window.height))
                cols = range(col_off, col_off + int(window.width))
                
                # Collect all attributes from the polygon
                attributes = poly.to_dict()
                
                # Append the rows and cols along with the polygon attributes to the data list
                for row in rows:
                    for col in cols:
                        pixel_data = {
                            'Pixel_Row': row,
                            'Pixel_Col': col,
                            **attributes  # This adds all polygon attributes
                        }
                        all_data.append(pixel_data)
        
        return pd.DataFrame(all_data)



gpkg_path = 'Datasets/niwot_aop_polygons_2023_12_8_23_analysis_ready_half_diam.gpkg'
existing_raster_path = "NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200801_161441_reflectance/NEON_D13_NIWO_DP1_20200801_161441_reflectanceNEON_D13_NIWO_DP1_20200801_161441_reflectance__envi"
processor = ENVIProcessor(existing_raster_path, gpkg_path)
df_polygons = processor.extract_data_by_polygons()
df_polygons

,Pixel_Row,Pixel_Col,GlobalID,CreationDate,Creator,EditDate,Editor,description_notes,dbh,tree_height,...,og_flight_date,collection_date,collector_name,plot,location,woody_shrub_height,imagery,combined_all_category_species,area_m,geometry
0,1222,554,{E9346797-777A-4D43-BD01-02A511C57DAA},2023-06-20 19:27:29+00:00,Tyler.L.McIntosh_ucboulder,2023-06-20T19:27:29+00:00,Tyler.L.McIntosh_ucboulder,None,None,None,...,2023-06-20 19:27:01+00:00,2023-06-20 19:26:55+00:00,Tyler,0,Brainard,None,AOP,Evergreen___Picea engelmannii,7.004149,POLYGON ((452521.07602110645 4435015.481140467...
1,1222,555,{E9346797-777A-4D43-BD01-02A511C57DAA},2023-06-20 19:27:29+00:00,Tyler.L.McIntosh_ucboulder,2023-06-20T19:27:29+00:00,Tyler.L.McIntosh_ucboulder,None,None,None,...,2023-06-20 19:27:01+00:00,2023-06-20 19:26:55+00:00,Tyler,0,Brainard,None,AOP,Evergreen___Picea engelmannii,7.004149,POLYGON ((452521.07602110645 4435015.481140467...
2,1223,554,{E9346797-777A-4D43-BD01-02A511C57DAA},2023-06-20 19:27:29+00:00,Tyler.L.McIntosh_ucboulder,2023-06-20T19:27:29+00:00,Tyler.L.McIntosh_ucboulder,None,None,None,...,2023-06-20 19:27:01+00:00,2023-06-20 19:26:55+00:00,Tyler,0,Brainard,None,AOP,Evergreen___Picea engelmannii,7.004149,POLYGON ((452521.07602110645 4435015.481140467...
3,1223,555,{E9346797-777A-4D43-BD01-02A511C57DAA},2023-06-20 19:27:29+00:00,Tyler.L.McIntosh_ucboulder,2023-06-20T19:27:29+00:00,Tyler.L.McIntosh_ucboulder,None,None,None,...,2023-06-20 19:27:01+00:00,2023-06-20 19:26:55+00:00,Tyler,0,Brainard,None,AOP,Evergreen___Picea engelmannii,7.004149,POLYGON ((452521.07602110645 4435015.481140467...
4,1224,558,{54D1A59F-2B5C-4D76-913A-690314E314A7},2023-06-20 19:28:37+00:00,Tyler.L.McIntosh_ucboulder,2023-06-20T19:28:37+00:00,Tyler.L.McIntosh_ucboulder,None,None,None,...,2023-06-20 19:27:01+00:00,2023-06-20 19:26:55+00:00,Tyler,0,Brainard,None,AOP,Non-vegetated & dead_Rock__,6.064980,POLYGON ((452524.7605621438 4435014.4842204545...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,808,548,{C3209290-3F79-4268-AE0F-4BE7F43E1C71},2023-06-20 21:04:00+00:00,Tyler.L.McIntosh_ucboulder,2023-06-20T21:04:00+00:00,Tyler.L.McIntosh_ucboulder,None,None,None,...,2023-06-20 21:03:43+00:00,2023-06-20 21:03:34+00:00,Katie,0,Brainard,10cm to 1m,AOP,Woody shrub_Woody shrub - Broadleaf__Unidentified,23.032479,"POLYGON ((452510.4188020002 4435431.162891659,..."
493,792,550,{56551F70-2BB6-4C7A-80A4-99CF53CD2EC3},2023-06-20 21:05:08+00:00,Tyler.L.McIntosh_ucboulder,2023-06-20T21:05:08+00:00,Tyler.L.McIntosh_ucboulder,None,None,None,...,2023-06-20 21:04:49+00:00,2023-06-20 21:04:42+00:00,Katie,0,Brainard,10cm to 1m,AOP,Woody shrub_Woody shrub - Broadleaf__Unidentified,7.422119,POLYGON ((452516.90320151206 4435445.897912507...
494,792,551,{56551F70-2BB6-4C7A-80A4-99CF53CD2EC3},2023-06-20 21:05:08+00:00,Tyler.L.McIntosh_ucboulder,2023-06-20T21:05:08+00:00,Tyler.L.McIntosh_ucboulder,None,None,None,...,2023-06-20 21:04:49+00:00,2023-06-20 21:04:42+00:00,Katie,0,Brainard,10cm to 1m,AOP,Woody shrub_Woody shrub - Broadleaf__Unidentified,7.422119,POLYGON ((452516.90320151206 4435445.897912507...
495,793,550,{56551F70-2BB6-4C7A-80A4-99CF53CD2EC3},2023-06-20 21:05:08+00:00,Tyler.L.McIntosh_ucboulder,2023-06-20T21:05:08+00:00,Tyler.L.McIntosh_ucboulder,None,None,None,...,2023-06-20 21:04:49+00:00,2023-06-20 21:04:42+00:00,Katie,0,Brainard,10cm to 1m,AOP,Woody shrub_Woody shrub - Broadleaf__Unidentified,7.422119,POLYGON ((452516.90320151206 4435445.897912507...


In [14]:
layer_columns = [f'Band_{i}' for i in range(1, 426)]
df_polygons = df_polygons.replace(-9999.0, np.nan)
# Drop rows where all values in the specified columns are NaN
df_cleaned_polygons = df_polygons.dropna(subset=layer_columns, how='any')
df_cleaned_polygons

,Pixel_Row,Pixel_Col,Band_1,GlobalID,CreationDate,Creator,EditDate,Editor,description_notes,dbh,...,Band_417,Band_418,Band_419,Band_420,Band_421,Band_422,Band_423,Band_424,Band_425,Band_426


In [24]:
import pandas as pd
from tqdm.auto import tqdm

def filter_large_csv_based_on_right_df_with_progress(large_csv_path, right_df, subset_columns, output_filtered_csv_path):
    """
    Filters rows in a large CSV file, keeping only those that match the subset_columns
    values found in the right_df. Writes the filtered rows to a new CSV file.
    Includes a progress bar to track processing progress.

    Parameters:
    - large_csv_path (str): Path to the large CSV file.
    - right_df (pd.DataFrame): Dataframe containing the filter criteria.
    - subset_columns (list of str): Columns used for filtering.
    - output_filtered_csv_path (str): Path to write the filtered CSV file.
    """
    # Get the unique combinations of subset_columns in right_df
    unique_combinations = right_df[subset_columns].drop_duplicates()

    # Convert the unique combinations to a set of tuples for faster searching
    unique_tuples = set([tuple(x) for x in unique_combinations.to_numpy()])

    # Determine total number of rows for progress bar
    total_rows = sum(1 for _ in open(large_csv_path, 'r', encoding='utf-8'))
    chunksize = 100000  # Adjust based on your memory constraints
    total_chunks = (total_rows // chunksize) + (1 if total_rows % chunksize else 0)

    # Initialize tqdm progress bar
    pbar = tqdm(total=total_chunks, desc='Processing CSV', unit='chunk')

    # Initialize a DataFrame to hold chunks that pass the filter
    filtered_chunks = []

    # Read the large CSV in chunks
    for chunk in pd.read_csv(large_csv_path, chunksize=chunksize):
        # Filter the chunk
        filtered_chunk = chunk[chunk.apply(lambda x: (x[subset_columns[0]], x[subset_columns[1]]) in unique_tuples, axis=1)]
        
        # If the filtered chunk is not empty, add it to the list
        if not filtered_chunk.empty:
            filtered_chunks.append(filtered_chunk)
        
        # Update progress bar
        pbar.update(1)

    pbar.close()

    # Concatenate all filtered chunks and write to the output CSV
    if filtered_chunks:
        filtered_df = pd.concat(filtered_chunks)
        filtered_df.to_csv(output_filtered_csv_path, index=False)
    else:
        print("No matching rows found in the large CSV.")

# Example usage

output_filtered_csv_path = 'NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200801_161441_reflectanc_polygons.csv'

filter_large_csv_based_on_right_df_with_progress(large_csv_path, right_df, subset_columns, output_filtered_csv_path)


Processing CSV:   0%|          | 0/66 [00:00<?, ?chunk/s]

In [29]:


import pandas as pd

def load_and_merge_csv_swapped(small_csv_path, other_df, merge_columns=['Pixel_Row', 'Pixel_Col'], how='inner'):
    """
    Loads a CSV file into a DataFrame and merges it with another DataFrame based on specified columns.
    In this version, the other DataFrame is treated as the left side of the merge.

    Parameters:
    - small_csv_path (str): Path to the CSV file to load.
    - other_df (pd.DataFrame): The other DataFrame to merge with the loaded DataFrame, treated as the left DataFrame.
    - merge_columns (list of str): Columns to merge on. Default is ['Pixel_Row', 'Pixel_Col'].
    - how (str): Type of merge to be performed. Default is 'inner'.

    Returns:
    - pd.DataFrame: The resulting merged DataFrame.
    """
    # Load the small CSV file into a DataFrame
    small_df = pd.read_csv(small_csv_path)

    # Merge the other DataFrame with the loaded DataFrame, treating other_df as the left DataFrame
    merged_df = pd.merge(other_df, small_df, on=merge_columns, how=how)

    return merged_df



# Perform the merge
merged_df = load_and_merge_csv_swapped('NIWOT_calibration_flight_08_2020/NEON_D13_NIWO_DP1_20200801_161441_reflectanc_polygons.csv', df_polygons)

# Optionally, you might want to view or save the resulting DataFrame
print(merged_df)
# merged_df.to_csv('path/to/your/merged_result.csv', index=False)


     Pixel_Row  Pixel_Col                                GlobalID  \
0         1222        554  {E9346797-777A-4D43-BD01-02A511C57DAA}   
1         1222        555  {E9346797-777A-4D43-BD01-02A511C57DAA}   
2         1223        554  {E9346797-777A-4D43-BD01-02A511C57DAA}   
3         1223        555  {E9346797-777A-4D43-BD01-02A511C57DAA}   
4         1224        558  {54D1A59F-2B5C-4D76-913A-690314E314A7}   
..         ...        ...                                     ...   
492        808        548  {C3209290-3F79-4268-AE0F-4BE7F43E1C71}   
493        792        550  {56551F70-2BB6-4C7A-80A4-99CF53CD2EC3}   
494        792        551  {56551F70-2BB6-4C7A-80A4-99CF53CD2EC3}   
495        793        550  {56551F70-2BB6-4C7A-80A4-99CF53CD2EC3}   
496        793        551  {56551F70-2BB6-4C7A-80A4-99CF53CD2EC3}   

                 CreationDate                     Creator  \
0   2023-06-20 19:27:29+00:00  Tyler.L.McIntosh_ucboulder   
1   2023-06-20 19:27:29+00:00  Tyler.L.McIntosh_u